Exploratory data analysis (EDA) is a crucial component of data science which allows one to understand the basics of what your data looks like and what kinds of questions might be answered by them. For this task, we are going to clean, sanitize and explore our data. Using the movies dataset, answer the following questions by writing code in the cells.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing the required packages here

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
import ast, json

from datetime import datetime
import matplotlib.pyplot as plt
% matplotlib inline


In [ ]:
#### load the movie dataset and create their dataframes

movies_df = pd.read_csv('/content/drive/My Drive/Hyperion/mod2t16/movies2.csv')


### Data Cleansing 
#### Clean the data. Identify columns that are redundant or unnecessary.

It is always easier to make your decisions based on data which is relevant and concise. Remove the following columns ['keywords', 'homepage', 'status', 'tagline', 'original_language', 'homepage', 'overview', 'production_companies', 'original_title', 'title_y'] from the data set as they will not be used in the analysis.

In [ ]:
movies_df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [ ]:
movies_df.drop(['keywords', 'homepage', 'status', 'tagline', 'original_language', 'homepage', 'overview', 'production_companies', 'original_title'],inplace=True, axis=1)

#### Remove any duplicate rows

In [ ]:
# seeing no duplicates
movies_df[movies_df.duplicated()]

# movies_df.drop_duplicates(keep=False,inplace=True)

,budget,genres,id,popularity,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count


#### Some movies in the database have zero budget or zero revenue which implies that their values have not been recorded or some information is missing. Discard such entries from the dataframe.

In [ ]:
movies_df = movies_df.loc[~((movies_df['budget'] == 0) | (movies_df['revenue'] == 0)),:]

#### To manipulate the columns easily, it is important that we make use of the python objects. Change the release date column into Date format and extract the year from the date. This will help us in analysing yearly data.

In [ ]:
movies_df.release_date.dtype

dtype('O')

In [ ]:
# create a new column, date_parsed, with the parsed dates
movies_df['date_parsed'] = pd.to_datetime(movies_df['release_date'], format='%Y-%M-%d')

movies_df['date_parsed'].head()

0   2009-01-10 00:12:00
1   2007-01-19 00:05:00
2   2015-01-26 00:10:00
3   2012-01-16 00:07:00
4   2012-01-07 00:03:00
Name: date_parsed, dtype: datetime64[ns]

In [ ]:
# Change the release_date column to DateTime column
movies_df['release_date'] = movies_df['date_parsed']

# Extract the release year from every release date
movies_df['release_year'] = movies_df['release_date'].dt.year

#### Change budget and revenue columns format to integer using numpy’s int64 method.

In [ ]:
# code here
movies_df[['budget','revenue']] = movies_df[['budget','revenue']].astype(np.int64)

On checking the dataset, we see that genres, keywords, production_companies, production_countries, spoken_languages are in the JSON format which will make it difficult to manipulate the dataframe. Now let’s flatten these columns into a format that can be easily interpreted.

I will help you convert them into strings and later convert them into lists. Writing a generic function to parse JSON columns.

In [ ]:
def parse_col_json(column, key):
    """
    Args:
        column: string
            name of the column to be processed.
        key: string
            name of the dictionary key which needs to be extracted
    """
    for index,i in zip(movies_df.index,movies_df[column].apply(json.loads)):
        list1=[]
        for j in range(len(i)):
            list1.append((i[j][key]))# the key 'name' contains the name of the genre
        movies_df.loc[index,column]=str(list1)
    
parse_col_json('genres', 'name')
parse_col_json('spoken_languages', 'name')
# parse_col_json('cast', 'name')
parse_col_json('production_countries', 'name')

movies_df.head()

,budget,genres,id,popularity,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,date_parsed,release_year
0,237000000,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",19995,150.437577,"['United States of America', 'United Kingdom']",2009-01-10 00:12:00,2787965087,162.0,"['English', 'Español']",Avatar,7.2,11800,2009-01-10 00:12:00,2009
1,300000000,"['Adventure', 'Fantasy', 'Action']",285,139.082615,['United States of America'],2007-01-19 00:05:00,961000000,169.0,['English'],Pirates of the Caribbean: At World's End,6.9,4500,2007-01-19 00:05:00,2007
2,245000000,"['Action', 'Adventure', 'Crime']",206647,107.376788,"['United Kingdom', 'United States of America']",2015-01-26 00:10:00,880674609,148.0,"['Français', 'English', 'Español', 'Italiano',...",Spectre,6.3,4466,2015-01-26 00:10:00,2015
3,250000000,"['Action', 'Crime', 'Drama', 'Thriller']",49026,112.312950,['United States of America'],2012-01-16 00:07:00,1084939099,165.0,['English'],The Dark Knight Rises,7.6,9106,2012-01-16 00:07:00,2012
4,260000000,"['Action', 'Adventure', 'Science Fiction']",49529,43.926995,['United States of America'],2012-01-07 00:03:00,284139100,132.0,['English'],John Carter,6.1,2124,2012-01-07 00:03:00,2012


### Now onto the exploration

#### Identify relationships between variables / features

The main goal here is to identify and create relationships which can help you to build ideas. I have defined questions which can help you identify some relationships to explore.

#### Which are the 5 most expensive movies? How do the most expensive and cheapest movies compare? Exploring the most expensive movies help you explore if some movies are worth the money spent on them based on their performance and revenue generated.

In [ ]:
print('The five biggest budget movies are : ')
print(movies_df.sort_values(by='budget', ascending=False)['title'].head())
print('\nThe five lowest budget movies are : ')
print(movies_df.sort_values(by='budget', ascending=True)['title'].head())


The five biggest budget movies are : 
17    Pirates of the Caribbean: On Stranger Tides
1        Pirates of the Caribbean: At World's End
7                         Avengers: Age of Ultron
10                               Superman Returns
4                                     John Carter
Name: title, dtype: object

The five lowest budget movies are : 
4238          Modern Times
3611    A Farewell to Arms
3372          Split Second
3419          Bran Nue Dae
4608          The Prophecy
Name: title, dtype: object


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
print(movies_df.sort_values(by='budget', ascending=False)[['popularity', 'vote_count', 'revenue', 'spoken_languages', 'production_countries']].head())

    popularity  vote_count     revenue                    spoken_languages  \
17  135.413856        4948  1045713802              ['English', 'Español']   
1   139.082615        4500   961000000                         ['English']   
7   134.279229        6767  1405403694                         ['English']   
10   57.925623        1400   391081192  ['English', 'Français', 'Deutsch']   
4    43.926995        2124   284139100                         ['English']   

            production_countries  
17  ['United States of America']  
1   ['United States of America']  
7   ['United States of America']  
10  ['United States of America']  
4   ['United States of America']  


In [ ]:
print(movies_df.sort_values(by='budget', ascending=True)[['popularity', 'vote_count', 'revenue', 'spoken_languages', 'production_countries']].head())

      popularity  vote_count  revenue spoken_languages  \
4238   28.276480         856  8500000      ['English']   
3611    1.199451          28       25      ['English']   
3372    4.857028          63        5      ['English']   
3419    0.050456           6        7               []   
4608   11.818333         138       16      ['English']   

              production_countries  
4238  ['United States of America']  
3611  ['United States of America']  
3372            ['United Kingdom']  
3419                            []  
4608  ['United States of America']  


#### What are the top 5 most profitable movies? Compare the min and max profits. The comparison helps us indentify the different approaches which failed and succeeded. Subtracting the budget from the revenue generated, will return the profit earned.

In [ ]:
# creating profit column
movies_df['profit'] = movies_df.revenue - movies_df.budget

# sorting by profit
print('The five most profitable movies are :')
print(movies_df.sort_values(by='profit', ascending=False)[['title']].head())
print('\nThe five least profitable movies are :')
print(movies_df.sort_values(by='profit', ascending=False)[['title']].tail())


The five most profitable movies are :
             title
0           Avatar
25         Titanic
28  Jurassic World
44       Furious 7
16    The Avengers

The five least profitable movies are :
                            title
311  The Adventures of Pluto Nash
208              The 13th Warrior
141               Mars Needs Moms
338                     The Alamo
13                The Lone Ranger


In [ ]:
# displaying other features of the most profitable movies
print(movies_df.sort_values(by='profit', ascending=False)[['title', 'profit', 'budget', 'revenue', 'popularity', 'vote_count', 'spoken_languages', 'production_countries']].head())

             title      profit     budget     revenue  popularity  vote_count  \
0           Avatar  2550965087  237000000  2787965087  150.437577       11800   
25         Titanic  1645034188  200000000  1845034188  100.025899        7562   
28  Jurassic World  1363528810  150000000  1513528810  418.708552        8662   
44       Furious 7  1316249360  190000000  1506249360  102.322217        4176   
16    The Avengers  1299557910  220000000  1519557910  144.448633       11776   

                                     spoken_languages  \
0                              ['English', 'Español']   
25  ['English', 'Français', 'Deutsch', 'svenska', ...   
28                                        ['English']   
44                                        ['English']   
16                                        ['English']   

                              production_countries  
0   ['United States of America', 'United Kingdom']  
25                    ['United States of America']  
28         

In [ ]:
# displaying other features of the least profitable movies
print(movies_df.sort_values(by='profit', ascending=False)[['title', 'profit', 'budget', 'revenue', 'popularity', 'vote_count', 'spoken_languages', 'production_countries']].tail())

                            title     profit     budget   revenue  popularity  \
311  The Adventures of Pluto Nash  -92896027  100000000   7103973   12.092241   
208              The 13th Warrior  -98301101  160000000  61698899   27.220157   
141               Mars Needs Moms -111007242  150000000  38992758   12.362599   
338                     The Alamo -119180039  145000000  25819961   10.660441   
13                The Lone Ranger -165710090  255000000  89289910   49.046956   

     vote_count        spoken_languages  \
311         142             ['English']   
208         510    ['English', 'Norsk']   
141         199             ['English']   
338         106  ['English', 'Español']   
13         2311             ['English']   

                          production_countries  
311  ['Australia', 'United States of America']  
208               ['United States of America']  
141               ['United States of America']  
338               ['United States of America']  
13       

#### Find the most talked about movies. Sort the dataframe based on the popularity column.

In [ ]:
# finding the most popular movies
print(movies_df.sort_values(by='popularity', ascending=False)[['title', 'popularity']].head())

                       title  popularity
546                  Minions  875.581305
95              Interstellar  724.247784
788                 Deadpool  514.569956
94   Guardians of the Galaxy  481.098624
127       Mad Max: Fury Road  434.278564


#### Find Movies which are rated above 7



In [ ]:
# finding movies rated above 7
movies_df[['title', 'vote_average']].loc[(movies_df['vote_average'] > 7)]

,title,vote_average
0,Avatar,7.2
3,The Dark Knight Rises,7.6
6,Tangled,7.4
7,Avengers: Age of Ultron,7.3
8,Harry Potter and the Half-Blood Prince,7.4
...,...,...
4713,Roger & Me,7.4
4724,Eraserhead,7.5
4738,Pi,7.1
4773,Clerks,7.4


In [ ]:
# finding movies rated above 7, ordered by rating
movies_df[['title', 'vote_average']].loc[(movies_df['vote_average'] > 7)].sort_values(by='vote_average', ascending=False)

,title,vote_average
1881,The Shawshank Redemption,8.5
2970,There Goes My Baby,8.5
3337,The Godfather,8.4
1818,Schindler's List,8.3
3232,Pulp Fiction,8.3
...,...,...
978,The Life Aquatic with Steve Zissou,7.1
1745,My Sister's Keeper,7.1
3436,The Messenger,7.1
1720,Kick-Ass,7.1


#### Which year did we have the most profitable movies?
This will help to raise some deeper questions about the years when movies didn’t do well. These are preliminary questions which give rise to questions providing deeper insights.

In [ ]:
# getting most profitable year(s)
most_profitable = movies_df[['release_year', 'profit']].groupby(['release_year']).sum().sort_values(by='profit', ascending=False)
print(f'The most profitable years for movies have been : \n {most_profitable}')

The most profitable years for movies have been : 
                    profit
release_year             
2014          17029736072
2012          16665370551
2015          16082841939
2013          15191240622
2009          13798015000
...                   ...
1929              3979000
1933              3842000
1935              2593000
1932                   21
1927            -91969578

[89 rows x 1 columns]


In [ ]:
print(f"The most profitable year for movies was {most_profitable.index[0]} with a profit of ${most_profitable.iloc[0].values[0]:,}")

The most profitable year for movies was 2014 with a profit of $17,029,736,072


#### Most successful genres — create a bar plot explaining the frequency of movies in each genre.

In [ ]:
# getting most profitable genres
genres_by_profit_df = movies_df[['genres', 'profit']].groupby(['genres']).sum().sort_values(by='profit', ascending=False).iloc[:10]

,profit
genres,
['Comedy'],10493600341
['Drama'],8002248589
"['Comedy', 'Romance']",6919367439
"['Adventure', 'Action', 'Science Fiction']",6296442640
"['Adventure', 'Fantasy', 'Action']",5902220721
"['Action', 'Adventure', 'Science Fiction']",5605734701
"['Animation', 'Family']",5366338646
"['Adventure', 'Action', 'Thriller']",4184897561
"['Drama', 'Romance']",3905785518
